created on: Tue Jan 14 09:44:36 2020
<br>
Group 7
<br>
@author: V.B., E.G.

In [210]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
import h5py
import warnings
warnings.filterwarnings('ignore')

# deep learning
import keras
from keras.models import load_model
from keras.applications.imagenet_utils import decode_predictions

In [196]:
project_path = 'C:/Users/emma.grandgirard/Documents/B - Projet Interpromo/'
models_path = project_path + 'Models/'
data_path = 'Data/data_Interpromo2020/Interpromo2020/All Data/ANALYSE IMAGE/'
path_out = project_path + 'pred'

# Choose social network: SEATGURU, SEATMAESTRO, INSTAGRAM
social_net = 'INSTAGRAM'
insta_hashtag = 'airbus'  # if social_net == 'INSTAGRAM'

# Choose images parameters
size = (224, 224)
greys = False

In [178]:
models_dirs = os.listdir(models_path)
models = dict({'View': 'model_view_13_01_VGG16.h5',
               'Int_man': 'model_int_man.h5',
               'Ext_typ': 'model_ext_typ.h5',  # missing
               'Int_Airbus': 'model_int_airbus.h5',
               'Int_Boeing': 'model_int_Boeing.h5'
})

### Read images to predict

In [7]:
def read_img(img_path: str, size: tuple, greys: bool=False) -> np.array:
    
    img_img = Image.open(img_path)
    img_arr = np.array(img_img.resize(size))
    img_arr = img_arr.reshape(1, size[0], size[1], 1 if greys else 3)
    
    return img_arr

In [12]:
# Path
if social_net == 'SEATGURU' or social_net == 'SEATMAESTRO':
    folder = 'IMG ' + social_net + '/'

elif social_net == 'INSTAGRAM':
    folder = social_net + '/' + insta_hashtag + '/'
    
path_pred = project_path + data_path + folder
imgs_names = os.listdir(path_pred)[:10]  # here, only 10 to test

# Init results DataFrame
df = pd.DataFrame(columns=['img', 'view', 'view_proba', 'manufacturer', 'manufacturer_proba', 'type', 'type_proba'])
df['img'] = imgs_names

In [23]:
# Read all images, convert to array
all_imgs_arr = np.array([read_img(path_pred + imgs_names[k], size=(size[0], size[1]), greys=False) 
                         for k in range(len(imgs_names))])  # len(imgs_names)

# Reshape for prediction
all_imgs_arr = all_imgs_arr.reshape(len(imgs_names), size[0], size[1], 1 if greys else 3)
all_imgs_arr.shape

(10, 224, 224, 3)

## Models pipeline

In [150]:
def pred_from_model(img_arr: np.array, models_path: str, model_name: str) -> str:
    
    # Load model and predict
    model = load_model(models_path + models[model_name])
    preds = model.predict(img_arr)

    # Add a line to get a list of labels (to be named 'classes')
    classes = ['Ext', 'Ext-Int', 'Int', 'Meal', 'Others']  # to modify
    labels = [np.argmax(preds[k]) for k in range(len(preds))]
    proba_labels = [np.max(preds[k]) for k in range(len(preds))]
    labels = [classes[labels[k]] for k in range(len(labels))]
    
    del model  # remove model from environment
    
    return labels, proba_labels

### Step 1: View

In [110]:
view_labels, view_proba_labels = pred_from_model(img_arr=all_imgs_arr, models_path=models_path, model_name='View')

In [136]:
# Fill DataFrame with View labels
df['view'] = view_labels
df['view_proba'] = view_proba_labels

### Step 2

#### Exterior --> Manufacturer and type

In [151]:
# If View == 'Ext', predict aircraft type
ind_ext = df[df['view'] == 'Ext'].index
ext_imgs_arr = all_imgs_arr[ind_ext]
#ext_labels, ext_proba_labels = pred_from_model(img_arr=ext_imgs_arr, models_path=models_path, model_name='Ext')

# Fake labels to test
ext_labels = ['A320', 'A330', '737', '747', 'A330', '737', 'A320']
ext_proba_labels = [0.99, 0.98, 0.87, 0.99, 0.98, 0.87, 0.6]

# Fill DataFrame with types and manufacturers
df['type'].loc[ind_ext] = ext_labels
df['type_proba'].loc[ind_ext] = ext_proba_labels
df['manufacturer'].loc[ind_ext] = ['Airbus' if 'A' in typ else 'Boeing' for typ in df['type'].loc[ind_ext]]

#### Interior --> Manufacturer

In [143]:
# If View == 'Int', predict manufacturer
ind_int = df[df['view'] == 'Int'].index
int_imgs_arr = all_imgs_arr[ind_int]
int_man_labels, int_man_proba_labels = pred_from_model(img_arr=int_imgs_arr, models_path=models_path, model_name='Int_man')

# Fake labels to test
int_man_labels = ['Boeing', 'Airbus']

# Fill DataFrame with types and manufacturers
df['manufacturer'].loc[ind_int] = int_man_labels
df['manufacturer_proba'].loc[ind_int] = int_man_proba_labels

### Step 3

In [184]:
# If View == 'Int' and manufacturer == 'Airbus', predict type
ind_int_Airbus = df[(df['view'] == 'Int') & (df['manufacturer'] == 'Airbus')].index
int_Airbus_imgs_arr = all_imgs_arr[ind_int_Airbus]
int_Airbus_labels, int_Airbus_proba_labels = pred_from_model(img_arr=int_Airbus_imgs_arr, models_path=models_path, 
                                                             model_name='Int_Airbus')

# Fake labels to test
int_Airbus_labels = ['A320']

# Fill DataFrame with types and manufacturers
df['type'].loc[ind_int_Airbus] = int_Airbus_labels
df['type_proba'].loc[ind_int_Airbus] = int_Airbus_proba_labels

In [194]:
# If View == 'Int' and manufacturer == 'Boeing', predict type
ind_int_Boeing = df[(df['view'] == 'Int') & (df['manufacturer'] == 'Boeing')].index
int_Boeing_imgs_arr = all_imgs_arr[ind_int_Boeing]
int_Boeing_labels, int_Boeing_proba_labels = pred_from_model(img_arr=int_Boeing_imgs_arr, models_path=models_path, 
                                                             model_name='Int_Boeing')

# Fake labels to test
int_Boeing_labels = ['737']

# Fill DataFrame with types and manufacturers
df['type'].loc[ind_int_Boeing] = int_Boeing_labels
df['type_proba'].loc[ind_int_Boeing] = int_Boeing_proba_labels

In [195]:
df

,img,view,view_proba,manufacturer,manufacturer_proba,type,type_proba
0,0.jpg,Ext,0.734326,Airbus,NaN,A320,0.99
1,1.jpg,Ext,0.998698,Airbus,NaN,A330,0.98
2,10.jpg,Ext,0.999712,Boeing,NaN,737,0.87
3,100.jpg,Ext,0.922856,Boeing,NaN,747,0.99
4,1000.jpg,Int,0.924982,Boeing,0.993151,737,0.956689
5,1001.jpg,Int,0.825127,Airbus,0.989164,A320,0.941314
6,1002.jpg,Ext,0.782494,Airbus,NaN,A330,0.98
7,1003.jpg,Meal,0.673317,NaN,NaN,NaN,NaN
8,1004.jpg,Ext,0.999980,Boeing,NaN,737,0.87
9,1005.jpg,Ext,0.837012,Airbus,NaN,A320,0.6


In [209]:
# Path to save
if social_net != 'INSTAGRAM':
    path_save = path_out + '/pred_' + social_net + '.csv'
    
else:
    path_save = path_out + '/pred_' + social_net + '_' + insta_hashtag + '.csv'

# Save
df.to_csv(path_or_buf=path_save, sep=';', encoding='utf-8')